Two-layer ocean climate model
========

In [ ]:
# Import required libraries.
import matplotlib.pyplot as plt
import numpy as np

import climate_model

In [ ]:
# Create an interactive climate model.
# Note, the settings here will be used in subsequent cells.
model = climate_model.ClimateModel()
ui = climate_model.ClimateModelUI(model)
ui.display()

In [ ]:
# Calculate the optimal lambda value.
# Note, this uses the values taken from the sliders for lam, d_d etc. (lam used as starting value).
rmse, opt_lam = model.optimize_lambda(**ui.get_values())
print(f'Optimal lambda is {opt_lam:.3f}, which gives an RMSE of {rmse:.6f}')

opt_control_values = ui.get_values()
opt_control_values['lam'] = opt_lam
# Run model with optimal lambda, and default values of all other variables.
model.run_model(**opt_control_values)
model.plot()

In [ ]:
# Some simple experiments with defined total forcings.
# Note, total_forcing's first year is 1750, so index 150 corresponds to 1900.
method = 'step'

if method == 'step':
    # Turn on forcing at 1900:
    total_forcing = np.zeros_like(model.forcings.year.loc[1:])
    total_forcing[150:] = 1
elif method == 'pulse':
    # Forcing pulse 1900-1901:
    total_forcing = np.zeros_like(model.forcings.year.loc[1:])
    total_forcing[150:152] = 3
elif method == '3xCO2':
    # 3x CO2 forcing:
    columns = [c for c in model.forcings.columns if c != 'year']
    forcings = model.forcings.copy()
    forcings['co2'] *= 3
    total_forcing = forcings.loc[1:][columns].sum(axis=1)
else:
    raise Exception(f'Unknown method: {method}')

# Note, this uses the values taken from the sliders for lambda, d_d etc.
model.run_model(total_forcing=total_forcing, **ui.get_values())
model.plot(show_obs=False)

In [ ]:
# Show results for a very simple ensemble of lambda values, from 0.4 to 1, with 5 steps.
# Does not use values from sliders.
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_size_inches((18, 8))
for lam in np.linspace(0.4, 1, 5):
    model.run_model(lam=lam)
    model.plot(ax1=ax1, ax2=ax2, show_forcing=False)
model.configure_axes([ax1, ax2])

In [ ]:
# Show IPCC-like uncertainty range for the 5 SSP scenarios, with a given uncertainty multiplier.
# Use the optimal lambda calculated above, using settings *when that cell was run*.
lam_uncert = 0.2
lam_min = opt_lam - opt_lam * lam_uncert
lam_max = opt_lam + opt_lam * lam_uncert

plt.figure(figsize=(20, 10))
for scenario in model.scenarios:
    new_control_values = opt_control_values.copy()
    new_control_values['future_scenario'] = scenario
    
    new_control_values['lam'] = lam_min
    model.run_model(**new_control_values)
    dTm_min = model.dT.dTm.values.copy()
    
    new_control_values['lam'] = lam_max
    model.run_model(**new_control_values)
    dTm_max = model.dT.dTm.values.copy()
    
    plt.fill_between(
        model.dT.year.values,
        dTm_min,
        dTm_max,
        alpha=0.4, 
        label=f'{scenario}'
    )
plt.legend(loc='upper left')
plt.xlabel('Year')
plt.ylabel('Scenario temperature anomaly 1850-1899 baseline (K)')
plt.xlim((2000, 2100))
plt.ylim((0, 8));